In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Load the data
train_df = pd.read_csv('data/train.csv')
val_df = pd.read_csv('data/val.csv')

# Prepare features and targets
X_train = train_df.drop(columns=['class']).values
y_train = train_df['class'].values
X_val = val_df.drop(columns=['class']).values
y_val = val_df['class'].values

# Step 1: One-hot encode the labels
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = onehot_encoder.fit_transform(y_train.reshape(-1, 1))
y_val_encoded = onehot_encoder.transform(y_val.reshape(-1, 1))

In [3]:
# Convert the data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_encoded, dtype=torch.float32)

In [4]:
# Step 2: Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # No activation on the last layer since we'll use CrossEntropyLoss
        return x

In [5]:
# Model parameters
input_size = X_train.shape[1]  # Number of keypoints (42 features, 21 x, y coordinates)
hidden_size = 128  # You can adjust this
output_size = y_train_encoded.shape[1]  # Number of classes

# Step 3: Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
def train(model, X_train_tensor, y_train_tensor, epochs=100):
    model.train()
    for epoch in range(epochs):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_train_tensor)
        
        # Compute the loss
        loss = criterion(outputs, torch.max(y_train_tensor, 1)[1])  # Use max to get class indices
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


In [12]:
train(model, X_train_tensor, y_train_tensor, epochs=2000)

Epoch [10/2000], Loss: 0.2076
Epoch [20/2000], Loss: 0.2057
Epoch [30/2000], Loss: 0.2028
Epoch [40/2000], Loss: 0.2008
Epoch [50/2000], Loss: 0.1984
Epoch [60/2000], Loss: 0.1958
Epoch [70/2000], Loss: 0.1939
Epoch [80/2000], Loss: 0.1914
Epoch [90/2000], Loss: 0.1888
Epoch [100/2000], Loss: 0.1868
Epoch [110/2000], Loss: 0.1849
Epoch [120/2000], Loss: 0.1828
Epoch [130/2000], Loss: 0.1802
Epoch [140/2000], Loss: 0.1779
Epoch [150/2000], Loss: 0.1757
Epoch [160/2000], Loss: 0.1737
Epoch [170/2000], Loss: 0.1714
Epoch [180/2000], Loss: 0.1693
Epoch [190/2000], Loss: 0.1668
Epoch [200/2000], Loss: 0.1645
Epoch [210/2000], Loss: 0.1625
Epoch [220/2000], Loss: 0.1601
Epoch [230/2000], Loss: 0.1578
Epoch [240/2000], Loss: 0.1563
Epoch [250/2000], Loss: 0.1537
Epoch [260/2000], Loss: 0.1515
Epoch [270/2000], Loss: 0.1495
Epoch [280/2000], Loss: 0.1478
Epoch [290/2000], Loss: 0.1459
Epoch [300/2000], Loss: 0.1440
Epoch [310/2000], Loss: 0.1419
Epoch [320/2000], Loss: 0.1401
Epoch [330/2000],

In [8]:
def evaluate(model, X_val_tensor, y_val_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(X_val_tensor)
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(y_val_tensor, 1)
        accuracy = accuracy_score(predicted.cpu(), labels.cpu())
    return accuracy


In [13]:
accuracy = evaluate(model, X_val_tensor, y_val_tensor)
print(f'Validation Accuracy: {accuracy:.2f}')

Validation Accuracy: 0.92


In [14]:
torch.save(model.state_dict(), 'mlp_hand_sign_classifier.pt')

In [16]:
loaded_model = MLP(42, 128, 26)

loaded_model.load_state_dict(torch.load('mlp_hand_sign_classifier.pt'))

<All keys matched successfully>

In [17]:
loaded_model.eval()

accuracy = evaluate(loaded_model, X_val_tensor, y_val_tensor)
print(f'Validation Accuracy of the loaded model: {accuracy:.2f}')

Validation Accuracy of the loaded model: 0.92
